# Advanced free energy analyses with SOMD

In this notebook we will explore advanced analysis techniques for alchemical free enregy calculations, using Sire tool `analyse_freenrg mbar`. 
Initially, you will learn how to correctly interpret phase space overlap matrix. Following, advanced commands will be shown to enhance the mbar analysis.

The notebook forms part of the CCPBio-Sim workshop **Alchemical Free Energy Simulation Analysis with analyse_freenrg** run on the 11th of April 2018 at the University of Bristol.

*Author: Anotnia Mey & Stefano Bosisio  
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: xx mins**

## Overlap matrix

In [ ]:
%pylab inline
import glob
import seaborn as sbn
sbn.set_style("ticks")
sbn.set_context("notebook", font_scale = 2)

### The overlap matrix

can be used to look at the phase space overlap of neighbouring lambdas. 
By adding the flag `--overlap` this matrix will be automatically computed and added to the output file. 

So let's look at the overlap matrix for a simulation of a host-guest system, shown in figure, obtained by running 16 $\lambda$ windows of 8 ns length each. 
This time we will write an output file called `good_overlap.dat`


<img src="images/CBC-G2.png" alt="host-guest" style="width: 300px;"/>


In [ ]:
%%capture run_info_good
#Let's run the analysis again with the keyword --overlap
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i good_overlap/lambda-*/simfile.dat -o good_overlap.dat --subsampling --overlap

In [ ]:
#A helper function to read the overlap matrix from file
def get_overlp_matrix(filename, lambda_val ):
    fh = open (filename, 'r')
    lines = fh.readlines()
    fh.close()
    count = 0
    matrix = []
    for line in lines:
        if line.startswith('#Overlap'):
            matrix = lines[(count+1):(count+1+lambda_vals)]
            break
        count = count+1 
    for i in range(len(matrix)):
        temp = matrix[i].strip().split(' ')
        float_temp = [float(j) for j in temp]
        matrix[i] = float_temp
    matrix =np.array(matrix)
    return matrix

In [ ]:
#an exercise could be add the lambda_vals in the function above
good_overlap = get_overlp_matrix('good_overlap.dat',16)


### Plotting the overlap matrix
The plotting library has a nice advanced heat map feature that allows you to not only plot a pictorial image of a matrix or heatmap but also add the numercal values making it easier to read the plot

In [ ]:
fig = figure(figsize=(12,12))
ax = sbn.heatmap(good_overlap, annot=True, fmt='.2f', linewidths=.5, annot_kws={"size": 12})
ax.set_xlabel(r'$\lambda$ index')
ax.set_ylabel(r'$\lambda$ index')
ax.set_title('Good overlap matrix')

### Example of a bad overlap matrix
Below we have the same simulation as before, but reducing the number of lambda windows from 11 to 6. What do you observe in terms of the overlap matrix?

Belowe we have the same simulation as before, but the number of $\lambda$ windows was reduced from 16 to 10. What do you observe in terms of the overlap matrix? 
Repeat the procedure above and create a file called `bad_overlap.dat`

In [ ]:
%%capture run_info_bad
#Let's run the analysis again with the keyword --overlap


In [ ]:
bad_overlap = get_overlp_matrix('bad_overlap.dat',10)

In [ ]:
fig = figure(figsize=(12,12))
ax = sbn.heatmap(good_overlap, annot=True, fmt='.2f', linewidths=.5, annot_kws={"size": 12})
ax.set_xlabel(r'$\lambda$ index')
ax.set_ylabel(r'$\lambda$ index')
ax.set_title('Bad overlap matrix')

### Advanced tasks

As an advanced task we are going to deal with mbar command `subsampling` and `discard`. The former option performs a subsamplin operation over all the samles data, written in the `simfile.dat` files. The second choice, `discard`, will discard a number of frames from the beginning of the simulation. This is beneficial for our estimation, as in the very first frames the system is usually equilibrating, giving rise to noise to the final free energy calculation. 

We are going to study a host-guest system, similar to the one shown before. To compute the binding free energy, the following thermodynamic cycle is adopted:
<img src="images/host_guest_cyclo.png" alt="host-guest" style="width: 500px;"/>
Initially, a `discharging` step is performed, where ligand's charges are turned off both in solvated and  bound phases. Following, a `vanishing` step is done, by  switching off ligand's Lennard Jones terms, in order to have a fully decouple molecule. 
The final free energy of binding is computed, by summing up the contribution of each leg of the cycle, as:
\begin{equation}
\Delta G_\mathrm{bind} = (\Delta G^\mathrm{solv}_\mathrm{elec} + \Delta G^\mathrm{solv}_\mathrm{vdW}) - (\Delta G^\mathrm{host}_\mathrm{elec} + \Delta G^\mathrm{host}_\mathrm{vdW} ) 
\end{equation}

Try to perform these steps:
1. Compute the discharging and vanishing free energy for the solvated and bound phase using the standard `analyse_freenrg mbar` command. Thus, retrieve the binding free energy using the equation above
2. Compute the discharging and vanishing free energy by adding the option `--subsampling` and discarding the first 500 frames (`discard 500`). What do you notice? What is happening to TI? and MBAR? What is the final binding free energy?
3. What conclusions can you draw from the previous points?


In [ ]:
#1. Compute the discharging and vanishing free energy for the solvated and bound phase:
#bound phase -discharging
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/discharge/output/lambda-*/simfile.dat -o bound_discharge.dat
#bound phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/vanish/output/lambda-*/simfile.dat -o bound_vanish.dat
#solvated phase -discharging
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/discharge/output/lambda-*/simfile.dat -o free_discharge.dat
#solvated phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/vanish/output/lambda-*/simfile.dat -o free_vanish.dat


In [ ]:
#helper function to extract DG from the freenrg_analysis generated files. 
def get_free_energy(ifile):
    reader = open(ifile,"r").readlines()
    mbar = float(reader[-3].split(",")[0])
    ti = float(reader[-1].split()[0])
    return mbar, ti

In [ ]:
#Extract the free energy changes from the output files
bound_discharge_mbar, bound_discharge_ti = get_free_energy('bound_discharge.dat')
bound_vanish_mbar,bound_vanish_ti = get_free_energy('bound_vanish.dat')

free_discharge_mbar,free_discharge_ti = get_free_energy('free_discharge.dat')
free_vanish_mbar, free_vanish_ti = get_free_energy('free_vanish.dat')

#Compute the free energy change in the bound and water phase for mbar
DG_bound_mbar =
DG_free_mbar = 
#Compute the free energy change in the bound and water phase for TI
DG_bound_ti = 
DG_free_ti = 


#Thus, using the equation above, compute the estimation of binding free energy using mbar and TI
DG_bind_mbar = DG_free_mbar - DG_bound_mbar 
DG_bind_ti =  

#What is the binding free energy with MBAR? and with TI?


In [ ]:
#3. Try to re run using the --discard option. What do you notice?
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/discharge/output/lambda-*/simfile.dat -o bound_discharge.dat --subsampling  --discard 500
#bound phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/vanish/output/lambda-*/simfile.dat -o bound_vanish.dat --subsampling  --discard 500
#solvated phase -discharging 
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/discharge/output/lambda-*/simfile.dat -o free_discharge.dat --subsampling  --discard 500
#solvated phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/vanish/output/lambda-*/simfile.dat -o free_vanish.dat --subsampling --discard 500



In [ ]:
#Extract the free enery change from the output files
bound_discharge_mbar, bound_discharge_ti = get_free_energy('bound_discharge.dat')
bound_vanish_mbar,bound_vanish_ti = get_free_energy('bound_vanish.dat')

free_discharge_mbar,free_discharge_ti = get_free_energy('free_discharge.dat')
free_vanish_mbar, free_vanish_ti = get_free_energy('free_vanish.dat')

#Compute the free energy change in the bound and water phase for mbar
DG_bound_mbar =
DG_free_mbar = 
#Compute the free energy change in the bound and water phase for TI
DG_bound_ti = 
DG_free_ti =


#Thus, using the equation above, compute the estimation of binding free energy using mbar and TI
DG_bind_mbar = 
DG_bind_ti =  

#What is the binding free energy with MBAR? and with TI?


The experimental standard binding free energy $\Delta G^\circ_\mathrm{bind}$ is 7.08 $\pm$ 0.01 kcal$\cdot$mol$^{-1}$.
Although our predictions are quite far apart, the subsampling and discard show an improvement of about  1 kcal$\cdot$mol$^{-1}$ on the final MBAR free energy estimation.  On the other side, TI fails to correctly predict the binding in this case. This is due to the noise present along the simulation, which greatly influences the free energy gradients values, thus the final free energy integration. Such a behaviour does not exist in MBAR, which is always consistent for all the evaluation process.
Finally, it is worth to mention that this is just the first step toward accurate predictions. What w ehave obtained here is just a binding free energy $\Delta G_\mathrm{bind}$ which cannot be compared to the experimental value. Indeed, we are missing the definition of a standard state for the simulation, thus a standard state correction term for the free energy evaluation. Additionally, we should consider also Lennard Jones corrections and electrostatic finite size artefacts correction, which can remarkably improve the final free energy estimation.

Congratulations you have finished this tutorial! Time for a coffee or tea break :-)